# Evaluation NER - échantillon en sortie d'HTR brut - CER 21% - WER 64%

In [1]:
import re

from bs4 import BeautifulSoup
import spacy
import srsly

## Fonctions utilisées pour les tests

In [2]:
def open_and_parse(source):
    '''open a file and parsed its content with BeautifulSoup'''
    with open(source, 'r', encoding='utf8') as fh:
        file_content = fh.read()
    parsed_source = BeautifulSoup(file_content, 'xml')
    return parsed_source


def split_word_on_caps(orig):
    '''
    Spots words stuck together and split them using regex
    
    e.g. thisIsAnExample --> this Is An Example
    '''
    
    # word segmentation using regex
    # regex are based on study of LECTAUREP's textual data and its recurrent errors
    letter_before_caps_splitted = re.sub(r"(\w)([A-Z])", r"\1 \2", orig)

    return letter_before_caps_splitted


def get_transcription_list(xml_tree):
    '''find text nodes in PAGE XML and get them ready for NER processing'''
    transcriptions = []
    col_5 = xml_tree.find_all('Unicode')
    
    for text in col_5:
        transcriptions.append(text.text)
    
    transcriptions_joined = '\n'.join(transcriptions)
    
    return transcriptions_joined

On récupère le texte qui servira à produire une prédiction pour le NER avec spaCy depuis un fichier PAGE XML. Pour chercher la transcription, on récupère le texte issus des balises ```<unicode>```. 

On obtient une liste qu'on normalise en reconstruisant des possibles mots aglutinés l'aide des majuscules.

On rassembles les index de la liste avec la méthode .join(), afin d'obtenir une chaîne de caractère en jointe avec des \n. 

# Création de la vérité de terrain NER

La vérité de terrain NER a été produite à l'aide de [Doccano](https://github.com/doccano/doccano). Le texte utilisé pour constituer une vérité de terrain est identique à celui utilisé pour la prédiction de NER. 

Cette méthodologie de test est plus adapté à l'évaluation d'un modèle entraîné pour une tâche spécifique. Cependant, il a été jugé pertinent de comparer ce qu'il était obtenu avec l'outil de NER brut, et ce qu'on attendrait de cet outil brut **au mieux** : la vérité de terrain. 

Le modèle de langue français utilisé avec spaCy permet de repérer 4 types d'entités : 
* `PER` : personne
* `LOC` : localisation
* `ORG` : organisations
* `MISC` : une entité générique indiquant une entité nommée, sans toutefois l'identifier.

Pour annoter le document, les résultats de NER obtenus avec spaCy ont été importés dans Doccano, puis ont été corrigés manuellement. 

La composition d'une vérité de terrain sur un échantillon en sortie d'HTR brut est particulier, car demandant d'annoter des mots bruités, cela s'apparente, à certains moments, presque à lire une autre langue.

Il a été décidé de se placer au niveau des tokens, et d'annoter si celui-ci aurait dû être détecté. Par exemple, les noms de personne arrivant systématiquement au début d'une rangée dans le document originel. 

Les corrections de l'annotation automatique concernent donc : 

* Les entités manquées
* La segmentation des entités
* L'identification des entités
* Suppression des "fausses" entités détectés lors de la prédiction

Les annotations manuelles respectent deux règles : 

* Les titres de civilités n'ont pas été relevés. 
* Les adresses ont été annotes dans leur forme complète, en incluant donc le mot "rue".

Pour générer une première annotation NER avec spaCy et importer le résultat dans un format adapté à Doccano, on utilise la classe suivante, écrite par Mary Chester-Kadwell et modifié pour les besoins de l'expérience. Voir les notebooks NER de [Mary Chester-Kadwell](https://github.com/mchesterkadwell/named-entity-recognition).

In [3]:
"""
All credits goes to https://github.com/mchesterkadwell/named-entity-recognition
See https://github.com/mchesterkadwell/named-entity-recognition/blob/main/doccano/doccano_named_ents.py

Modified version of Mary Chester-Kadwell's custom class to generate and output spaCy NER labels in Doccano format from
XML input. (Github https://github.com/mchesterkadwell)

The class has been modified to extract data with a french language model from an XMLPage input, in order to quickly
establish a ground truth for NER for benchmarking, but not for training a model.
"""

from pathlib import Path
import json
from bs4 import BeautifulSoup
import fr_core_news_lg

nlp = fr_core_news_lg.load(disable=['tagger', 'parser'])


class DoccanoNamedEnts:
    def __init__(self, xml_folder_path):
        self.xml_path = xml_folder_path
        self.spacy_ents = self._init_spacy_ents()
        self.doccano_ents = self._init_doccano_ents()

    def print(self):
        """Prints all texts and their named entities in Doccano format, encode in UTF8 and decode the string."""
        for item in self.doccano_ents:
            text_first = dict(sorted(item.items(), reverse=True))
            text_first_utf8 = json.dumps(text_first, ensure_ascii=False).encode('utf8')
            print(text_first_utf8.decode())

    def to_file(self, output_file_path):
        """Writes all texts and their named entities in Doccano format to specified file."""
        file = Path(output_file_path).absolute()
        with file.open('w+', encoding='utf-8') as f:
            for item in self.doccano_ents:
                item_ = item.copy()
                item_json = json.dumps(item_, ensure_ascii=False).encode('utf8')
                f.write(item_json.decode())
            print(f'{len(self.doccano_ents)} items saved to {output_file_path}')

    def _init_spacy_ents(self):
        """Returns NER results in spaCy format from the initialised texts."""
        spacy_t = tuple(self.label_spacy_ents(text) for text in self.get_text())
        return spacy_t

    def _init_doccano_ents(self):
        """Returns NER results in Doccano format from the initialised texts."""
        doccano_t = tuple(self.convert_ents(ents) for ents in self.spacy_ents)
        return doccano_t

    def get_text(self):
        """Returns a text extracted from an XMLPage file, and normalize it."""
        with open(self.xml_path) as file:
            transcription = BeautifulSoup(file, "lxml-xml")
        parsed_transcription = transcription.find_all("Unicode")
        transcription_list = [i.text for i in parsed_transcription]
        processed_list_text = [split_word_on_caps(i) for i in transcription_list]
        text = ['\n'.join(processed_list_text)]
        return text

    @classmethod
    def convert_ents(cls, spacy_ne):
        """Converts NER results in spaCy format to Doccano format."""
        doccano_d = spacy_ne.copy()
        doccano_d['labels'] = doccano_d.pop('ents')
        new_labels = []
        for item in list(doccano_d['labels']):
            values = list(item.values())
            new_labels.append(values)
        doccano_d['labels'] = new_labels
        return doccano_d

    @classmethod
    def label_spacy_ents(cls, text):
        """Returns NER results predicted by spaCy for a single text."""
        doc = nlp(text)
        doc_dict = doc.to_json()
        return {key: value for (key, value) in doc_dict.items() if key in ['text', 'ents']}


In [4]:
labels = DoccanoNamedEnts("../../corpus_test/lectaurep/doc_28_sample/HTR_cer_21/FRAN_0025_0029_L-0.xml")
labels.print()

labels.to_file('spacy_ner_to_doccano/doc28_sample_cer_21.jsonl')

{"text": "An 1901, mois de Étévrier\nDupuis par Pierre) à Paris, rue Turot 4, à Catherine Bigol, safes\na Oo (par Mve) s.n. à\nMuret (de Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine\nLemaire, dt à Paris, même adreré (Sléparation de biens\nDuboscq et pitre de 320f- de rente 36 au nom de. Debuison. Juatine Adèle\nVe de Snles) et autres, à Paris, rue del'Arcade 20\nRourfpar Ddmond Marcitse Bt à Paris, rue Cavalotti 15, etautresspr toucher a reaoir\nChabr (par Marie Louise Jatoune, Ve de Marie Eusébe Maseine Bt à Billancourt ronte de\nSersailles. 18, à Ougène Vahrin et Marceline Fhalino, dt à Billancourt, rue Mationale 31, de 21303.88\nGauther (par Marie Cexandinte) à Paris, rue St Ferdinandt, à sespère et mère\nOussonppar Geordes Ernert Léou dt à Paris, rue Lamarch 144, à sonpère\nLecrosnier (etlivret de Caistte d'Epargne de Paris, de157.19, aunonde Pierre\nPugute) décédé en sondomicilé à P Paris, rue Descartes 21, le 16 8bre 1900\

Une fois le jsonl obtenu, on peut compléter et corriger la préannotation dans Doccano.

## Traitement du texte à plat, sans structure logique

 On charge le [modèle de langue française](https://spacy.io/models/fr#fr_core_news_lg) de spaCy.

In [5]:
nlp = spacy.load("fr_core_news_lg")

# Traitement de l'échantillon testé

On ouvre et on parse l'échantillon que l'on teste.

In [6]:
source_ner_eval = "../../corpus_test/lectaurep/doc_28_sample/HTR_cer_21/FRAN_0025_0029_L-0.xml"
content_ner_eval = open_and_parse(source_ner_eval)
sample_ner_eval = get_transcription_list(content_ner_eval)
sample_splitted_ner_eval = split_word_on_caps(sample_ner_eval)

print(sample_splitted_ner_eval)

An 1901, mois de Étévrier
Dupuis par Pierre) à Paris, rue Turot 4, à Catherine Bigol, safes
a Oo (par Mve) s.n. à
Muret (de Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine
Lemaire, dt à Paris, même adreré (Sléparation de biens
Duboscq et pitre de 320f- de rente 36 au nom de. Debuison. Juatine Adèle
Ve de Snles) et autres, à Paris, rue del'Arcade 20
Rourfpar Ddmond Marcitse Bt à Paris, rue Cavalotti 15, etautresspr toucher a reaoir
Chabr (par Marie Louise Jatoune, Ve de Marie Eusébe Maseine Bt à Billancourt ronte de
Sersailles. 18, à Ougène Vahrin et Marceline Fhalino, dt à Billancourt, rue Mationale 31, de 21303.88
Gauther (par Marie Cexandinte) à Paris, rue St Ferdinandt, à sespère et mère
Oussonppar Geordes Ernert Léou dt à Paris, rue Lamarch 144, à sonpère
Lecrosnier (etlivret de Caistte d'Epargne de Paris, de157.19, aunonde Pierre
Pugute) décédé en sondomicilé à P Paris, rue Descartes 21, le 16 8bre 1900
Lettéronctlespiénond 

On passe la vérité de terrain dans la chaîne de traitement TAL.

In [7]:
doc = nlp(sample_splitted_ner_eval)

## Traitement de la vérité de terrain

On ouvre le fichier jsonl grâce à la librairie srsly et la méthode read_jsonl().

On récupère la vérité de terrain créée avec Doccano dans un dictionnaire sous le format suivant :

```
{entities: [(start, end, type)]}
```

Ce dictionnaire nous servira à évaluer les performances du modèle générique.

In [22]:
filepath = '../../corpus_test/lectaurep/doc_28_sample/HTR_cer_21/NER_ground_truth/gt.jsonl'
# On utilise le package srsly de spaCy pour transformer un fichier JSONL -format d'export de Doccano- en une liste de dictionnaires
annotations = list(srsly.read_jsonl(filepath))

In [23]:
annotations

[{'id': 8,
  'data': "An 1901, mois de Étévrier\nDupuis par Pierre) à Paris, rue Turot 4, à Catherine Bigol, safes\na Oo (par Mve) s.n. à\nMuret (de Paul Louis Georges) dt à Paris, Bd Rochechouart 68, et Marie Joséphine\nLemaire, dt à Paris, même adreré (Sléparation de biens\nDuboscq et pitre de 320f- de rente 36 au nom de. Debuison. Juatine Adèle\nVe de Snles) et autres, à Paris, rue del'Arcade 20\nRourfpar Ddmond Marcitse Bt à Paris, rue Cavalotti 15, etautresspr toucher a reaoir\nChabr (par Marie Louise Jatoune, Ve de Marie Eusébe Maseine Bt à Billancourt ronte de\nSersailles. 18, à Ougène Vahrin et Marceline Fhalino, dt à Billancourt, rue Mationale 31, de 21303.88\nGauther (par Marie Cexandinte) à Paris, rue St Ferdinandt, à sespère et mère\nOussonppar Geordes Ernert Léou dt à Paris, rue Lamarch 144, à sonpère\nLecrosnier (etlivret de Caistte d'Epargne de Paris, de157.19, aunonde Pierre\nPugute) décédé en sondomicilé à P Paris, rue Descartes 21, le 1

In [24]:
gt_data = {}
for annotation in annotations :
    gt_entities = {'entities': [(start, end, type_) for start, end, type_ in annotation.get('label')]}
    text_gt = annotation.get('data')

In [25]:
gt_entities

{'entities': [(0, 7, 'MISC'),
  (17, 27, 'MISC'),
  (28, 34, 'PER'),
  (39, 45, 'PER'),
  (50, 55, 'LOC'),
  (73, 88, 'PER'),
  (119, 124, 'PER'),
  (129, 147, 'PER'),
  (155, 160, 'LOC'),
  (162, 177, 'LOC'),
  (185, 201, 'PER'),
  (217, 222, 'LOC'),
  (261, 268, 'PER'),
  (310, 318, 'PER'),
  (320, 334, 'PER'),
  (362, 367, 'LOC'),
  (369, 386, 'LOC'),
  (396, 411, 'PER'),
  (418, 423, 'LOC'),
  (472, 477, 'PER'),
  (483, 503, 'PER'),
  (539, 550, 'LOC'),
  (560, 570, 'LOC'),
  (579, 593, 'PER'),
  (597, 614, 'PER'),
  (622, 633, 'LOC'),
  (665, 672, 'LOC'),
  (678, 694, 'PER'),
  (699, 704, 'LOC'),
  (706, 723, 'LOC'),
  (784, 789, 'LOC'),
  (820, 830, 'PER'),
  (844, 870, 'ORG'),
  (890, 903, 'PER'),
  (934, 941, 'ORG'),
  (1003, 1014, 'PER'),
  (1039, 1056, 'LOC'),
  (1073, 1079, 'PER'),
  (1170, 1176, 'LOC'),
  (1182, 1196, 'PER'),
  (1329, 1335, 'LOC'),
  (1378, 1386, 'PER'),
  (1392, 1412, 'PER'),
  (1420, 1425, 'LOC'),
  (1461, 1477, 'PER'),
  (1484, 1489, 'LOC'),
  (1529, 154

## Vérification de l'alignement entre l'échantillon testé pour la prédiction NER et le texte utilisé pour la vérité de terrain NER

Pour vérifier l'alignement des deux textes utilisés pour la comparaison, on utilise un assert sur les chaînes de caractère de la vérité de terrain et de l'échantillon annotée.

In [26]:
assert text_gt == sample_splitted_ner_eval

On peut également tester l'alignement des tokens.

In [27]:
# On teste les tokens
assert [t.text for t in nlp(text_gt)] == [t.text for t in doc]

## Evaluation

On crée un objet Example contenant normalement des informations pour l'entraînement. Il sera utilisé pour stocker le document traité par la pipeline NLP et donc les prédictions de NER, ainsi que la vérité de terrain. 

Voir :
* https://spacy.io/api/example
* https://spacy.io/api/data-formats#dict-input
* https://spacy.io/api/scorer#score_spans

In [28]:
from spacy.training import Example

example = Example.from_dict(doc, gt_entities)

/home/hugoscheithauer/anaconda3/envs/ner_spacy/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "An 1901, mois de Étévrier
Dupuis par Pierre) à ..." with entities "[(0, 7, 'MISC'), (17, 27, 'MISC'), (28, 34, 'PER')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


On instancie l'objet Scorer.


In [29]:
from spacy.scorer import Scorer

scorer = Scorer()
# On récupère les scores.
scores = scorer.score_spans([example], 'ents')

In [30]:
scores

{'ents_p': 0.4689265536723164,
 'ents_r': 0.5123456790123457,
 'ents_f': 0.48967551622418887,
 'ents_per_type': {'ORG': {'p': 0.16666666666666666,
   'r': 0.125,
   'f': 0.14285714285714288},
  'PER': {'p': 0.65, 'r': 0.48148148148148145, 'f': 0.553191489361702},
  'LOC': {'p': 0.4077669902912621,
   'r': 0.5915492957746479,
   'f': 0.4827586206896552},
  'MISC': {'p': 0.125, 'r': 0.5, 'f': 0.2}}}

On peut procéder autrement avec :

In [31]:
from spacy.language import Language

scores_alt = nlp.evaluate([example])

scores_alt

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'pos_acc': None,
 'morph_acc': None,
 'morph_per_feat': None,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': 0.47701149425287354,
 'ents_r': 0.5123456790123457,
 'ents_f': 0.494047619047619,
 'ents_per_type': {'MISC': {'p': 0.125, 'r': 0.5, 'f': 0.2},
  'LOC': {'p': 0.4158415841584158,
   'r': 0.5915492957746479,
   'f': 0.4883720930232558},
  'ORG': {'p': 0.16666666666666666, 'r': 0.125, 'f': 0.14285714285714288},
  'PER': {'p': 0.6610169491525424,
   'r': 0.48148148148148145,
   'f': 0.5571428571428572}},
 'tag_acc': None,
 'lemma_acc': None,
 'speed': 7000.136916326263}

In [32]:
len_gt_entities = len(gt_entities['entities'])
len_prediction_entities = len(doc.ents)
print(f"nombre d'entités dans la vérité de terrain NER : {len_gt_entities} - nombre d'entités dans la prédiction NER : {len_prediction_entities}")

nombre d'entités dans la vérité de terrain NER : 166 - nombre d'entités dans la prédiction NER : 177


## Visualisation de la prédiction NER

In [33]:
from spacy import displacy

html = displacy.render([doc], style='ent', page=True)